# 빅프 9조 데이터분석

필요 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
import time

1m에 해당하는 위경도차(살짝안맞음 수정필요)

In [2]:
NS_DIFF_1M = 0.000009
EW_DIFF_1M = 0.000009

순찰 반경과 격자점 간격(m단위)

In [38]:
PATROL_HR = 900
GRID_DIFF = 100

파출소 위경도

In [4]:
police = np.array([37.655030, 127.066941])

grids에서 쓸 함수 격자 정보(간격 개수, 간격사이 크기) 생성

In [5]:
def grid_infos(hr, diff):
    grid_count = 2*hr//diff + 1
    ns_diff_grid = 2*hr/(grid_count-1)*NS_DIFF_1M
    ew_diff_grid = 2*hr/(grid_count-1)*EW_DIFF_1M
    return grid_count, np.array([ns_diff_grid, ew_diff_grid], dtype=np.float64)

원형으로 깎기

In [49]:
def filtering(police, grid):
    ft = grid - police
    ft[:,0]/=NS_DIFF_1M
    ft[:,1]/=EW_DIFF_1M
    ft**=2
    ft=np.sum(ft,axis=1)
    ft**=0.5
    grid = grid[ft <= PATROL_HR,:]
    return grid

격자점 배열 생성

In [50]:
def grids(police, hr, diff):
    grid_count, grid_info=grid_infos(hr, diff)
    start = [police[i] - grid_count//2 * grid_info[i] for i in range(2)]
    result = np.full((grid_count, grid_count, 2), start[0], dtype=np.float64)
    result[:,:,1] = start[1]
    
    interval = np.arange(result.shape[0], dtype=np.float64)
    interval_t = interval.reshape(interval.shape[0], 1).copy()
    interval *= grid_info[0]
    interval_t *= grid_info[1]
    
    result[:,:,0] += interval_t
    result[:,:,1] += interval
    
    return filtering(police, result.reshape(-1,2))

지도 객체 생성

In [51]:
m = folium.Map(location = police, zoom_start=60)

지도에 격자점 표시

In [52]:
def draw_marker(m, grid):
    folium.CircleMarker(location=grid[grid.shape[0]//2],
                        radius=10, color='#ff0000', fill=True).add_to(m)
    for i in grid:
        folium.CircleMarker(location=i, radius=10, color='#000000').add_to(m)

실행부

In [53]:
police_grid = grids(police, PATROL_HR, GRID_DIFF)
draw_marker(m, police_grid)

지도 표시

In [54]:
m

### 위경도->주소

변환서비스 https://www.geoservice.co.kr/

이용을 위해 격자점 배열을 csv로 저장

In [11]:
grid_df = pd.DataFrame(police_grid, columns=["위도","경도"])

In [29]:
name = "police_grid"
grid_df.to_csv(name + ".csv", index = False, encoding='cp949')

생성된 csv 변환서비스에 넣고 돌리기

돌린 데이터의 순서는 grid_df와 같다

위,경도 도로명 주소만 뽑기

In [32]:
addr_data = pd.read_csv("result_address.csv")
addr_data = addr_data.loc[:,["위도", "경도", "_ROAD_AD"]]
addr_data.columns = ["위도", "경도", "주소"]
addr_data["유형"] = "None"
addr_data

,위도,경도,주소,유형
0,37.64603,127.057941,서울특별시 노원구 덕릉로 443,None
1,37.64603,127.058841,서울특별시 노원구 덕릉로 459-21,None
2,37.64603,127.059741,서울특별시 노원구 덕릉로 459-21,None
3,37.64603,127.060641,서울특별시 노원구 덕릉로 459-18,None
4,37.64603,127.061541,서울특별시 노원구 덕릉로 459-18,None
...,...,...,...,...
436,37.66403,127.072341,서울특별시 노원구 한글비석로36길 59,None
437,37.66403,127.073241,서울특별시 노원구 한글비석로24바길 16,None
438,37.66403,127.074141,서울특별시 노원구 한글비석로24자길 5,None
439,37.66403,127.075041,서울특별시 노원구 수락산로 12,None


### 주소->건물이름

검색 안정화 위한 지연상수 (초 단위)

밑 함수에서 에러뜨면 지연 늘릴것

In [21]:
DELAY_TIME=1

셀레니움 구동 위한 웹브라우저 설정

In [22]:
options = wb.ChromeOptions()
options.add_argument('headless')
driver = wb.Chrome(options=options)
#driver = wb.Chrome()
driver.get('https://map.naver.com/')
time.sleep(DELAY_TIME)
inputTag = driver.find_element(By.CLASS_NAME, value='input_search')


입력창 초기화, 주소 검색

In [23]:
def search_loc(address):
    try:
        clear = driver.find_element(By.CLASS_NAME, 'btn_clear')
        clear.click()
    except:
        pass
    finally:
        inputTag.send_keys(address)
        inputTag.send_keys('\n')

건물 이름 찾아 반환 이름 안뜨면 noname처리

In [24]:
def build_name():
    try:
        build = driver.find_element(By.CLASS_NAME, value='build_name')
    except:
        return "noname"
    else:
        return build.text

검색 실행

In [25]:
def find_build(address):
    inputTag.clear()
    try:
        search_loc(address)
    except:
        return "nosearch"
    else:
        time.sleep(DELAY_TIME)
        return build_name()

크롤링 후 유형에 넣기

오래 걸림 -> DELAY_TIME 때문 최적의 상수 찾아야함

In [ ]:
for i in addr_data.index:
    addr_data.iloc[i, 3] = find_build(addr_data.iloc[i, 2])

In [ ]:
print(sorted(set(addr_data["유형"])))

In [ ]:
addr_data.to_csv("노원역지구대격자점.csv", index = False, encoding='cp949')